In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

C:\Program Files\Anaconda3\envs\schoolds\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California h

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [5]:
X_train.sort_values(by='CRIM')

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
284,0.00906,90.0,2.97,0.0,0.400,7.088,20.8,7.3073,1.0,285.0,15.3,394.72,7.85
285,0.01096,55.0,2.25,0.0,0.389,6.453,31.9,7.3073,1.0,300.0,15.3,394.72,8.23
341,0.01301,35.0,1.52,0.0,0.442,7.241,49.3,7.0379,1.0,284.0,15.5,394.74,5.49
55,0.01311,90.0,1.22,0.0,0.403,7.249,21.9,8.6966,5.0,226.0,17.9,395.93,4.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,45.74610,0.0,18.10,0.0,0.693,4.519,100.0,1.6582,24.0,666.0,20.2,88.27,36.98
410,51.13580,0.0,18.10,0.0,0.597,5.757,100.0,1.4130,24.0,666.0,20.2,2.60,10.11
405,67.92080,0.0,18.10,0.0,0.693,5.683,100.0,1.4254,24.0,666.0,20.2,384.97,22.98
418,73.53410,0.0,18.10,0.0,0.679,5.957,100.0,1.8026,24.0,666.0,20.2,16.45,20.62


2. Обучите стандартную регрессию, а также Ridge и Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [6]:
from sklearn import linear_model as lm

lin_reg = lm.LinearRegression()
fit_lin_reg = lin_reg.fit(X_train, y_train)

ridge_reg = lm.Ridge(RANDOM_STATE)
fit_ridge_reg = ridge_reg.fit(X_train, y_train)

lasso_reg = lm.Lasso(RANDOM_STATE)
fit_lasso_reg = lasso_reg.fit(X_train, y_train)

from sklearn.metrics import r2_score
print('Значение метрики r2 (коэффициент детерминации) для модели линейной регрессии: ' + str(r2_score(y_test, lin_reg.predict(X_test))))
print('Значение метрики r2 (коэффициент детерминации) для модели Ridge: ' + str(r2_score(y_test, ridge_reg.predict(X_test))))
print('Значение метрики r2 (коэффициент детерминации) для модели LASSO: ' + str(r2_score(y_test, lasso_reg.predict(X_test))))

Значение метрики r2 (коэффициент детерминации) для модели линейной регрессии: 0.7064403227878493
Значение метрики r2 (коэффициент детерминации) для модели Ridge: 0.7244072760384892
Значение метрики r2 (коэффициент детерминации) для модели LASSO: 0.13228963154072448


In [7]:
# pd.DataFrame(ridge_reg.coef_, X_train.columns, columns=['coef']).sort_values(by='coef', ascending=False)

3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
alphas = list(np.geomspace(10**-5, 10**5, 11))

grid_search_cv_ridge = (GridSearchCV(lm.Ridge(), {'alpha': alphas}, scoring='r2')).fit(X_train, y_train)
grid_search_cv_lasso = (GridSearchCV(lm.Lasso(random_state=RANDOM_STATE), {'alpha': alphas}, scoring='r2')).fit(X_train, y_train)

print(grid_search_cv_ridge.best_estimator_)
print(grid_search_cv_lasso.best_estimator_)
print('Метрика r2 (коэффициент детерминации) по модели Ridge с кросс-валидацией и лучшим коэффициентом регуляризации: \r\n'
      + str(grid_search_cv_ridge.best_estimator_.score(X=X_test, y=y_test)))
print('Метрика r2 (коэффициент детерминации) по модели LASSO с кросс-валидацией и лучшим коэффициентом регуляризации: \r\n'
      + str(grid_search_cv_lasso.best_estimator_.score(X=X_test, y=y_test)))

Ridge(alpha=0.1)
Lasso(alpha=0.0001, random_state=42)
Метрика r2 (коэффициент детерминации) по модели Ridge с кросс-валидацией и лучшим коэффициентом регуляризации: 
0.7087090635048612
Метрика r2 (коэффициент детерминации) по модели LASSO с кросс-валидацией и лучшим коэффициентом регуляризации: 
0.7064933046514985


### Вывод
Модель по Ridge изменилась незначительно, по большей части уменьшился, вероятно, из-за наличия кросс-валидации
Модель по LASSO улучшилась, благодаря подобранному параметру.

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
scalers = [MinMaxScaler(), StandardScaler()]
models = [lm.Ridge(), lm.Lasso(random_state=RANDOM_STATE)]
from itertools import product

scal_mods = list(product(scalers, models))

print(*list((map(lambda x: 'Метрика r2 на масштабированной ('
                              + str(x[0]).split('(')[0] + ') выборке по методу '
                              + str(x[1]).split('(')[0] + ': '
                              + str((Pipeline([('scaler', x[0]), ('model', x[1])]).fit(X_train, y_train)).score(X_test, y_test))
                    , scal_mods))), sep='\r\n')

Метрика r2 на масштабированной (MinMaxScaler) выборке по методу Ridge: 0.7070845645507637
Метрика r2 на масштабированной (MinMaxScaler) выборке по методу Lasso: 0.2533189827893051
Метрика r2 на масштабированной (StandardScaler) выборке по методу Ridge: 0.7065344206975689
Метрика r2 на масштабированной (StandardScaler) выборке по методу Lasso: 0.6492888897832787


# Вывод
MinMaxScaler отработал лучше при использовании Ridge и лучше в сравнение с StandardScaler.
При использовании Lasso лучше масштабирование было не использовать. Метрика только ухудшилась.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [11]:
for scaler, model in scal_mods:
    X_train_scaled = scaler.fit_transform(X_train, y_train)
    X_test_scaled = scaler.fit_transform(X_test, y_test)
    print('Результат метрики r2 при подборе коэффициента регуляризации для масштабирования '
          + str(scaler).split('(')[0]
          + ' и модели '
          + str(model).split('(')[0]
          + ': '
          + str((GridSearchCV(model, {'alpha': alphas}, scoring='r2').fit(X_train_scaled, y_train)).best_estimator_.score(X=X_test_scaled, y=y_test)))

Результат метрики r2 при подборе коэффициента регуляризации для масштабирования MinMaxScaler и модели Ridge: 0.48795625790832453
Результат метрики r2 при подборе коэффициента регуляризации для масштабирования MinMaxScaler и модели Lasso: 0.44364915632256885
Результат метрики r2 при подборе коэффициента регуляризации для масштабирования StandardScaler и модели Ridge: 0.6707509520722196
Результат метрики r2 при подборе коэффициента регуляризации для масштабирования StandardScaler и модели Lasso: 0.669271600528933


In [12]:
print(*list(map(lambda x: 'Результат метрики r2 при подборе коэффициента регуляризации для масштабирования '
                          + str(x[0]).split('(')[0]
                          + ' и модели '
                          + str(x[1]).split('(')[0]
                          + ': '
                          + str((GridSearchCV(x[1], {'alpha': alphas}, scoring='r2').fit(x[0].fit_transform(X_train, y_train), y_train))
                                .best_estimator_
                                .score(X=x[0].fit_transform(X_test, y_test), y=y_test)), scal_mods)), sep='\r\n')

Результат метрики r2 при подборе коэффициента регуляризации для масштабирования MinMaxScaler и модели Ridge: 0.48795625790832453
Результат метрики r2 при подборе коэффициента регуляризации для масштабирования MinMaxScaler и модели Lasso: 0.44364915632256885
Результат метрики r2 при подборе коэффициента регуляризации для масштабирования StandardScaler и модели Ridge: 0.6707509520722196
Результат метрики r2 при подборе коэффициента регуляризации для масштабирования StandardScaler и модели Lasso: 0.669271600528933


# Вывод
При использовании масштабирования везде идет прирост значения метрики r2, кроме случая применения масштабирования MinMaxScaler для модели Ridge

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [13]:
from sklearn.preprocessing import PolynomialFeatures
for scaler, model in scal_mods:
    X_train_scaled = scaler.fit_transform(X_train, y_train)
    X_test_scaled = scaler.fit_transform(X_test, y_test)
    X_train_poly = PolynomialFeatures(degree=(1, 2)).fit_transform(X_train_scaled, y_train)
    X_test_poly = PolynomialFeatures(degree=(1, 2)).fit_transform(X_test_scaled, y_test)
    print('Метрика r2 на масштабируемой выборке ('
          + str(scaler).split('(')[0]
          + ') с применением полиномиальных признаков для модели '
          + str(model).split('(')[0]
          + ': '
          + str(r2_score(y_test, (model.fit(X_train_poly, y_train)).predict(X_test_poly))))

Метрика r2 на масштабируемой выборке (MinMaxScaler) с применением полиномиальных признаков для модели Ridge: 0.7213499736691886
Метрика r2 на масштабируемой выборке (MinMaxScaler) с применением полиномиальных признаков для модели Lasso: 0.2596801657985459
Метрика r2 на масштабируемой выборке (StandardScaler) с применением полиномиальных признаков для модели Ridge: 0.4112684529743279
Метрика r2 на масштабируемой выборке (StandardScaler) с применением полиномиальных признаков для модели Lasso: 0.7404276060048538


# Вывод:
Использование полиномиальных признаков существенно повысило метрику r2

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [14]:
%%time
import warnings
warnings.filterwarnings('ignore')

from itertools import combinations
poly_degree = list(combinations(list(range(5)), 2))

scal_mods_poly = list(product(scalers, models, poly_degree))

list_params = []
for scaler, model, poly in scal_mods_poly:
    X_train_scaled = scaler.fit_transform(X_train, y_train)
    X_test_scaled = scaler.fit_transform(X_test, y_test)
    X_train_poly = PolynomialFeatures(degree=poly).fit_transform(X_train_scaled, y_train)
    X_test_poly = PolynomialFeatures(degree=poly).fit_transform(X_test_scaled, y_test)
    grid_search_be = GridSearchCV(model, {'alpha': alphas}, scoring='r2').fit(X_train_poly, y_train).best_estimator_
    list_params.append([scaler, grid_search_be, poly, grid_search_be.score(y=y_test, X=X_test_poly)])
list_params
df_result = pd.DataFrame(list_params, columns=['scaler', 'model', 'polynomial', 'r2'])

Wall time: 2min 26s


In [15]:
pd.Series(df_result[df_result['r2'] == df_result['r2'].max()].iloc[0]).rename('value')

scaler                             MinMaxScaler()
model         Lasso(alpha=0.001, random_state=42)
polynomial                                 (3, 4)
r2                                       0.794119
Name: value, dtype: object

http://archive.ics.uci.edu/ml/datasets/Adult

In [16]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'

import os
import requests as rq

if not os.path.exists(os.path.abspath(os.path.curdir) + '\\data'):
    print('Cоздаем отсутствующую папку для хранения данных')
    os.mkdir(os.path.abspath(os.path.curdir) + '\\data')
if not os.path.exists(os.path.abspath(os.path.curdir) + '\\data\\adult-all.csv'):
    print('Скачиваем файл: ' + link)
    with open(os.path.abspath(os.path.curdir) + '\\data\\adult-all.csv', 'wb') as f:
        f.write((rq.get(link)).content)
        f.close

8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [17]:
data = pd.read_csv(os.path.abspath(os.path.curdir) + '\\data\\adult-all.csv', header=None)

data['result'] = np.nan
data['result'].iloc[data[data[14] == '>50K'].index.to_list()] = 1
data['result'].iloc[data[data[14] == '<=50K'].index.to_list()] = 0
data = data.rename(columns={0: 'age', 1: 'company_type', 3: 'education', 5: 'married', 6: 'position', 7: 'family_status', 8: 'skin_color', 9: 'sex', 13: 'country', 14: 'salary'})
data_X = data.drop(columns=['result', 'salary'])
data_y = data[['result']]
data_X.head()

,age,company_type,2,education,4,married,position,family_status,skin_color,sex,10,11,12,country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [18]:
from sklearn.impute import SimpleImputer

ls_col_nulls = []
for key in data_X.keys().tolist():
    ls_col_nulls.append([key, data[data_X[key].isna()][key].count()])

df_cols_nulls = pd.DataFrame(ls_col_nulls)

if df_cols_nulls[1].sum() == 0: print('Пропуски в данных отсутствуют')
else:
    data_X = pd.DataFrame((SimpleImputer(missing_values=None, strategy='most_frequent').fit_transform(data_X)), columns=data_X.keys().tolist())

Пропуски в данных отсутствуют


10. Выберите колонки с числовыми и категориальными переменными.

In [19]:
ls_cat = []
ls_int = []
for i in data_X.keys().to_list():
    # print(str(i) + ': ' + str(data_X[i].nunique()), end='; ')
    if data_X[i].nunique() <= 10: ls_cat.append(i)
    if data_X[i].dtype == np.dtype(np.int64): ls_int.append(i)
print('')
print(ls_cat)
print(ls_int)


['company_type', 'married', 'family_status', 'skin_color', 'sex']
['age', 2, 4, 10, 11, 12]


11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [20]:
from sklearn.preprocessing import OneHotEncoder
data_X_class = pd.merge(left=pd.DataFrame(Pipeline(steps=[('cat', OneHotEncoder())]).fit_transform(data_X[ls_cat]).toarray()).reset_index()
                              , right=pd.DataFrame(Pipeline([('scaler', MinMaxScaler())]).fit_transform(data_X[ls_int])).reset_index()
                              , on='index', ).set_index('index')

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [21]:
ls_bin_cat = []
for i in data_X.keys().to_list():
    if data_X[i].nunique() == 2: ls_bin_cat.append(i)

ls_max_cat = []
for i in ls_cat:
    value, quantity = (data[[i]].groupby(by=[i])[[i]].count().rename(columns={i: 'quantity'})).sort_values(by='quantity', ascending=False).reset_index().iloc[0].to_list()
    ls_max_cat.append([i, value, quantity])

ls_max = pd.DataFrame(ls_max_cat).max().to_list()

from sklearn.metrics import f1_score, accuracy_score

print('f1_score: ' + str(f1_score(data_y, (data_X[ls_max[0]] == ls_max[1]).astype(int))))
print('accuracy_score: ' + str(accuracy_score(data_y, (data_X[ls_max[0]] == ls_max[1]).astype(int))))


f1_score: 0.3969017193960598
accuracy_score: 0.3400147414110806


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [22]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score

data_X_class_train, data_X_class_test, data_y_class_train, data_y_class_test = train_test_split(data_X_class, data_y, test_size=0.2, train_size=0.8)

for model in [LogisticRegression(), SVC(), LinearSVC()]:
    print(model)
    estim = model.fit(data_X_class_train, data_y_class_train)
    print('    ' + str(accuracy_score(data_y_class_test, estim.predict(data_X_class_test))))
    print('    f1_score: ' + str(f1_score(data_y_class_test, estim.predict(data_X_class_test))))
    svc = cross_val_score(model, data_X_class, data_y)
    print('    cross_val_score: ' + str(svc))
    print('    cross_val_score: ' + str(svc.mean()))

LogisticRegression()
    0.8474767120483161
    f1_score: 0.6500704556129638
    cross_val_score: [0.8431774  0.84307503 0.84961097 0.84438984 0.84479934]
    cross_val_score: 0.845010518991991
SVC()
    0.8452246903470161
    f1_score: 0.6321167883211678
    cross_val_score: [0.83979937 0.84225612 0.8482801  0.84152334 0.84520885]
    cross_val_score: 0.8434135533275671
LinearSVC()
    0.8474767120483161
    f1_score: 0.6454069490718706
    cross_val_score: [0.8431774  0.8463507  0.84817772 0.8458231  0.84531122]
    cross_val_score: 0.8457680276801989
Wall time: 6min 43s


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [23]:
data_edited = data.copy()
for i in data_edited.keys().to_list(): data_edited[i].iloc[data[data[i] == '?'].index.to_list()] = None

data_edited = pd.DataFrame((SimpleImputer(missing_values=None, strategy='most_frequent').fit_transform(data_X)), columns=data_X.keys().tolist())

data_X_edited = pd.merge(left=pd.DataFrame(Pipeline(steps=[('cat', OneHotEncoder())]).fit_transform(data_edited[ls_cat]).toarray()).reset_index()
                              , right=pd.DataFrame(Pipeline([('scaler', MinMaxScaler())]).fit_transform(data_edited[ls_int])).reset_index()
                              , on='index', ).set_index('index')

for i in data_edited.keys().to_list():
    if type(data_edited[i].iloc[0]) != type(''): data_edited[i] = data_edited[i].astype(int)

data_X_edited_train, data_X_edited_test, data_y_edited_train, data_y_edited_test = train_test_split(
    data_X_edited
    , data_y
    , test_size=0.2
    , train_size=0.8)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [24]:
%%time
for model in [LogisticRegression(), SVC(), LinearSVC()]:
    print(model)
    estim = model.fit(data_X_edited_train, data_y_edited_train)
    print('    ' + str(accuracy_score(data_y_edited_test, estim.predict(data_X_edited_test))))
    print('    f1_score: ' + str(f1_score(data_y_edited_test, estim.predict(data_X_edited_test))))
    svc = cross_val_score(model, data_X_edited, data_y)
    print('    cross_val_score: ' + str(svc))
    print('    cross_val_score: ' + str(svc.mean()))

LogisticRegression()
    0.8538233186610708
    f1_score: 0.6651031894934334
    cross_val_score: [0.8431774  0.84307503 0.84961097 0.84438984 0.84479934]
    cross_val_score: 0.845010518991991
SVC()
    0.8478861705394616
    f1_score: 0.636852394916911
    cross_val_score: [0.83979937 0.84225612 0.8482801  0.84152334 0.84520885]
    cross_val_score: 0.8434135533275671
LinearSVC()
    0.8529020370559934
    f1_score: 0.6569587013607067
    cross_val_score: [0.8431774  0.8463507  0.8482801  0.8458231  0.84531122]
    cross_val_score: 0.845788502700674
Wall time: 6min 23s


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [25]:
%%time
data_drop = data.copy()
for i in data_drop.keys().to_list(): data_drop[i].iloc[data[data[i] == '?'].index.to_list()] = None
# data_edited[['result']]
# data_drop.info()
data_dropped = data_drop.dropna(axis=0)

data_all_dropped = pd.merge(left=pd.DataFrame(Pipeline(steps=[('cat', OneHotEncoder())]).fit_transform(data_dropped[ls_cat]).toarray()).reset_index()
                              , right=pd.DataFrame(Pipeline([('scaler', MinMaxScaler())]).fit_transform(data_dropped[ls_int])).reset_index()
                              , on='index', ).set_index('index')

data_X_dropped_train, data_X_dropped_test, data_y_dropped_train, data_y_dropped_test = train_test_split(
    data_all_dropped
    , data_dropped[['result']]
    , test_size=0.2
    , train_size=0.8)

for model in [LogisticRegression(), SVC(), LinearSVC()]:
    print(model)
    est = model.fit(data_X_dropped_train, data_y_dropped_train)
    print('    ' + str(accuracy_score(data_y_dropped_test, est.predict(data_X_dropped_test))))
    print('    f1_score: ' + str(f1_score(data_y_dropped_test, est.predict(data_X_dropped_test))))
    svc = cross_val_score(model, data_all_dropped, data_dropped[['result']])
    print('    cross_val_score: ' + str(svc))
    print('    cross_val_score: ' + str(svc.mean()))

LogisticRegression()
    0.8389165284687673
    f1_score: 0.6415744157441574
    cross_val_score: [0.8379215  0.83957988 0.84674923 0.83834586 0.8404467 ]
    cross_val_score: 0.8406086355289257
SVC()
    0.8353786622443339
    f1_score: 0.6173220251863275
    cross_val_score: [0.83637369 0.83869541 0.8449801  0.83569217 0.83945157]
    cross_val_score: 0.8390385875917967
LinearSVC()
    0.8399115533443892
    f1_score: 0.6394422310756972
    cross_val_score: [0.83803206 0.84344942 0.84509067 0.84111013 0.84066785]
    cross_val_score: 0.8416700247350402
Wall time: 5min 27s


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [26]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

for clf in [RandomForestClassifier(), GradientBoostingClassifier()]:
    cvs = cross_val_score(clf, data_X_edited, data_y)
    print(str(clf) + ': ' + str(cvs) + '; mean = ' + str(cvs.mean()))

RandomForestClassifier(): [0.84716962 0.84440577 0.84981572 0.84561835 0.84377559]; mean = 0.8461570111508692
GradientBoostingClassifier(): [0.86160303 0.86242195 0.86762899 0.86302211 0.86506962]; mean = 0.8639491395377361


18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [27]:
%%time

X = data_edited.copy()
df_y = data_y

ls_cat = []
ls_int = []
for i in X.keys().to_list():
    if X[i].nunique() <= 10: ls_cat.append(i)
    if X[i].dtype == np.dtype(np.int): ls_int.append(i)

models = [LogisticRegression(), SVC(), LinearSVC(), RandomForestClassifier(), GradientBoostingClassifier()]
cat = [OneHotEncoder()]
scalers = [MinMaxScaler(), StandardScaler()]

list_params = []

for categ, scaler in list(product(cat, scalers)):
    df_X = pd.merge(left=pd.DataFrame(Pipeline(steps=[('cat', categ)]).fit_transform(X[ls_cat]).toarray()).reset_index()
                    , right=pd.DataFrame(Pipeline([('scaler', scaler)]).fit_transform(X[ls_int])).reset_index()
                    , on='index').set_index('index')
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, train_size=0.8)
    for model in models:
        print(model)
        estim = model.fit(X_train, y_train)
        accur = accuracy_score(y_test, estim.predict(X_test))
        f1 = f1_score(y_test, estim.predict(X_test))
        cvs = cross_val_score(model, df_X, df_y)
        print('    f1:       ' + str(f1))
        print('    accuracy: ' + str(accur))
        print('    cvs:      ' + str(cvs))
        print('    cvs.mean: ' + str(cvs.mean()))
    list_params.append([categ, scaler, model, f1, accur, cvs.mean()])
df_res = pd.DataFrame(list_params, columns=['categorial', 'scaler', 'model', 'f1', 'accuracy', 'cvs_mean'])

LogisticRegression()
    f1:       0.6270346779900919
    accuracy: 0.8381615313747569
    cvs:      [0.8431774  0.84307503 0.84961097 0.84438984 0.84479934]
    cvs.mean: 0.845010518991991
SVC()
    f1:       0.6106685976345643
    accuracy: 0.8348858634455932
    cvs:      [0.83979937 0.84225612 0.8482801  0.84152334 0.84520885]
    cvs.mean: 0.8434135533275671
LinearSVC()
    f1:       0.6296472831267874
    accuracy: 0.8409253761899887
    cvs:      [0.8431774  0.8463507  0.8482801  0.8458231  0.84531122]
    cvs.mean: 0.845788502700674
RandomForestClassifier()
    f1:       0.6642841188295735
    accuracy: 0.8461459719520934
    cvs:      [0.84553178 0.84368922 0.8499181  0.8474611  0.84428747]
    cvs.mean: 0.8461775343774525
GradientBoostingClassifier()
    f1:       0.6723809523809524
    accuracy: 0.8591462790459617
    cvs:      [0.86160303 0.86242195 0.86762899 0.86302211 0.86506962]
    cvs.mean: 0.8639491395377361
LogisticRegression()
    f1:       0.6442658875091307
    a

In [28]:
df_res[df_res['cvs_mean'] == df_res['cvs_mean'].min()].iloc[0]

categorial                                      OneHotEncoder()
scaler                                           MinMaxScaler()
model         ([DecisionTreeRegressor(criterion='friedman_ms...
f1                                                     0.672381
accuracy                                               0.859146
cvs_mean                                               0.863949
Name: 0, dtype: object